In [1]:
import pickle
import sklearn

In [2]:
# Read the data
with open(f"../data/model_data.pickle", 'rb') as f:
    model_data = pickle.load(f)

In [3]:
# Drop rows where days_since_led is < 0
model_data = model_data[model_data['days_since_led'] >= 0]

# Drop rows were surprise is NaN
model_data = model_data[model_data['surprise'] > -1000]

In [16]:
rsi_threshold = 
rsi = model_data['rsi'].values <= rsi_threshold
rsi_lag1 = model_data['rsi_lag1'].values <= rsi_threshold
rsi_lag2 = model_data['rsi_lag2'].values > rsi_threshold
rsi_data = model_data[rsi & rsi_lag1 & rsi_lag2]

In [10]:
rsi_data.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'symbol', 'led',
       'surprise', 'days_since_led', 'rsi', 'rsi_lag1', 'rsi_lag2',
       'num_stdev', 'num_stdev_lag1', 'num_stdev_lag2', '3wk_adj_return'],
      dtype='object')

In [ ]:
import matplotlib.pyplot as plt

In [23]:
for i in range(0,35):
    rsi_threshold = i
    rsi = model_data['rsi'].values <= rsi_threshold
    rsi_lag1 = model_data['rsi_lag1'].values <= rsi_threshold
    rsi_lag2 = model_data['rsi_lag2'].values > rsi_threshold
    rsi_data = model_data[rsi & rsi_lag1 & rsi_lag2]

    print(f"RSI Threshold {i}: {len(rsi_data)} cases, mean return {rsi_data['3wk_adj_return'].mean()}")
#     plt.scatter(rsi_data['volume'], rsi_data['3wk_adj_return'])
#     plt.xlim([0,2])
#     plt.xlabel('Volume')
#     plt.ylim([-20,20])
#     plt.ylabel(f'% Gain Relative to S&P - RSI t = {i}')

#     plt.show()

RSI Threshold 0: 0 cases, mean return nan
RSI Threshold 1: 0 cases, mean return nan
RSI Threshold 2: 0 cases, mean return nan
RSI Threshold 3: 2 cases, mean return 1.269993610972205
RSI Threshold 4: 3 cases, mean return 0.9005023689813417
RSI Threshold 5: 9 cases, mean return -0.5141198857335088
RSI Threshold 6: 27 cases, mean return 3.7744960962264105
RSI Threshold 7: 54 cases, mean return -1.7586246459407413
RSI Threshold 8: 94 cases, mean return -1.520046704285901
RSI Threshold 9: 141 cases, mean return -4.05503771016566
RSI Threshold 10: 228 cases, mean return -3.500480222053573
RSI Threshold 11: 327 cases, mean return -2.9499828376063655
RSI Threshold 12: 453 cases, mean return -2.7701610273550017
RSI Threshold 13: 602 cases, mean return -1.9648284816502062
RSI Threshold 14: 818 cases, mean return -1.7661833323488896
RSI Threshold 15: 1037 cases, mean return -1.6874332545278212
RSI Threshold 16: 1298 cases, mean return -1.549008247112833
RSI Threshold 17: 1628 cases, mean return -

In [26]:
import numpy as np
for i in np.arange(-3.0,-1.0, 0.1):
    std_threshold = i
    std = model_data['num_stdev'].values <= std_threshold
    std_lag1 = model_data['num_stdev_lag1'].values <= std_threshold
    std_lag2 = model_data['num_stdev_lag2'].values > std_threshold
    std_data = model_data[std & std_lag1 & std_lag2]

    print(f"STD Threshold {i}: {len(std_data)} cases, mean return {std_data['3wk_adj_return'].mean()}")

STD Threshold -3.0: 0 cases, mean return nan
STD Threshold -2.9: 0 cases, mean return nan
STD Threshold -2.8: 0 cases, mean return nan
STD Threshold -2.6999999999999997: 0 cases, mean return nan
STD Threshold -2.5999999999999996: 0 cases, mean return nan
STD Threshold -2.4999999999999996: 0 cases, mean return nan
STD Threshold -2.3999999999999995: 0 cases, mean return nan
STD Threshold -2.2999999999999994: 0 cases, mean return nan
STD Threshold -2.1999999999999993: 1 cases, mean return -1.97647261174522
STD Threshold -2.099999999999999: 26 cases, mean return -0.332551966905527
STD Threshold -1.9999999999999991: 98 cases, mean return -1.8011293352501556
STD Threshold -1.899999999999999: 328 cases, mean return -1.0730122614410518
STD Threshold -1.799999999999999: 727 cases, mean return -1.2469305357760068
STD Threshold -1.6999999999999988: 1459 cases, mean return -1.4108429441220018
STD Threshold -1.5999999999999988: 2546 cases, mean return -1.416252600183689
STD Threshold -1.49999999999

In [ ]:
is_NaN = model_data.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = model_data[row_has_NaN]

In [ ]:
# Standardize the data

# Drop rows where days_since_led is < 0
model_data = model_data[model_data['days_since_led'] >= 0]

# Drop rows were surprise is NaN
model_data = model_data[model_data['surprise'] >= 0]

# Generate label
def gen_label(x):
    if x > 7:
        return 1
    else:
        return 0
labels = [gen_label(x) for x in model_data['3wk_adj_return'].values]

# Drop cols we don't want for modeling
model_data = model_data.drop(columns=['date', 'open', 'high', 'low', 'close', 'symbol', 'led', '3wk_adj_return'])

# Scale rsi
model_data['rsi'] = model_data['rsi'] / 100
model_data['rsi_lag1'] = model_data['rsi_lag1'] / 100
model_data['rsi_lag2'] = model_data['rsi_lag2'] / 100

In [ ]:
hidden_layers = (10,7,5)

In [ ]:
model = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=hidden_layers,
                                            activation='relu',
                                            max_iter=200)

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(model_data, labels, test_size=.3, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
def bool_flag(x):
    if x==1:
        return True
    else:
        return False


In [ ]:
model.score(filtered_x, [0]*len(filtered_x))

In [ ]:
# only1_filter = [bool_flag(x) for x in y_train]
# only1_x = X_train[only1_filter]
# model.fit(only1_x, [1]*len(only1_x))

label_filter = [not bool_flag(x) for x in y_test]
only1_x_test = X_test[label_filter]
model.score(only1_x_test, [0]*len(only1_x_test))

In [ ]:
len(only1_x_test)

In [ ]:
hidden_layers = (10,7,5)

In [ ]:
model = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=hidden_layers,
                                            activation='relu',
                                            max_iter=200)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(model_data, labels, test_size=.3, random_state=42)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)